<p style="font-size:32px;text-align:center"> <b>Social network Graph Link Prediction - Facebook Challenge</b> </p>

>>>>>>>>>>>>> # Featurization

In [1]:
import pickle
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import networkx as nx


# 1. Reading Data

In [2]:
X_train = pd.read_csv('datasets/my/xtrain.csv')
Y_train = pd.read_csv('datasets/my/ytrain.csv')
X_test = pd.read_csv('datasets/my/xtest.csv')
Y_test = pd.read_csv('datasets/my/ytest.csv')
Y_train.shape,X_train.shape,Y_test.shape,X_test.shape

((15100029, 1), (15100029, 2), (3775007, 1), (3775007, 2))

graph created using the positive training data as the negative data consists of unconnected edges which is useless while creating graph

In [3]:
tg=nx.read_edgelist('datasets/data/after_eda/train_pos_after_eda.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int) 

# 2. Similarity measures

## 2.1 Jaccard Distance:
http://www.statisticshowto.com/jaccard-index/

\begin{equation}
j = \frac{|X\cap Y|}{|X \cup Y|} 
\end{equation}

### 2.1.1 For Followee (i.e. the source and destination nodes are Followee)

In [4]:
def jacc_followee(u,v,g):
    try:
        u_followers  = set([ x for (x,y) in g.in_edges(u)])
        v_followers  = set([ x for (x,y) in g.in_edges(v)])
        if len(u_followers)==0 | len(v_followers)==0:
            return 0
        return len(u_followers.intersection(v_followers))/len(u_followers.union(v_followers))
    except:
        return 0 

### 2.1.2 For Follower (i.e. the source and destination nodes are Followers)

In [5]:
def jacc_follower(u,v,g):
    try:
        u_followees  = set([ y for (x,y) in g.out_edges(u)])
        v_followees  = set([ y for (x,y) in g.out_edges(v)])
        if len(u_followees)==0 | len(v_followees)==0:
            return 0
        return len(u_followees.intersection(v_followees))/len(u_followees.union(v_followees))
    except:
        return 0 

### 2.1.3 AAI Functions

In [6]:
# #for followees
# def jaccard_for_followees(a,b):
#     try:
#         if len(set(train_graph.successors(a))) == 0  | len(set(train_graph.successors(b))) == 0:
#             return 0
#         sim = (len(set(train_graph.successors(a)).intersection(set(train_graph.successors(b)))))/\
#                                     (len(set(train_graph.successors(a)).union(set(train_graph.successors(b)))))
#     except:
#         return 0
#     return sim
    
# #for followers
# def jaccard_for_followers(a,b):
#     try:
#         if len(set(train_graph.predecessors(a))) == 0  | len(set(g.predecessors(b))) == 0:
#             return 0
#         sim = (len(set(train_graph.predecessors(a)).intersection(set(train_graph.predecessors(b)))))/\
#                                  (len(set(train_graph.predecessors(a)).union(set(train_graph.predecessors(b)))))
#         return sim
#     except:
#         return 0

## 2.2 Cosine distance

\begin{equation}
CosineDistance = \frac{|X\cap Y|}{|X|\cdot|Y|} 
\end{equation}
it has an

In [7]:
# u,v being followee
def cosine_followee(u,v,g):
    try:
        u_followers  = set([ x for (x,y) in g.in_edges(u)])
        v_followers  = set([ x for (x,y) in g.in_edges(v)])
        if len(u_followers)==0 | len(v_followers)==0:
            return 0
        return len(u_followers.intersection(v_followers))/math.sqrt(len(u_followers)*len(v_followers))
    except:
        return 0 
# u,v being followers
def cosine_follower(u,v,g):
    try:
        u_followees  = set([ y for (x,y) in g.out_edges(u)])
        v_followees  = set([ y for (x,y) in g.out_edges(v)])
        if len(u_followees)==0 | len(v_followees)==0:
            return 0
        return len(u_followees.intersection(v_followees))/math.sqrt(len(u_followees)*len(v_followees))
    except:
        return 0 

### AAI functions

In [8]:
# #for followees
# def cosine_for_followees(a,b):
#     try:
#         if len(set(tg.successors(a))) == 0  | len(set(tg.successors(b))) == 0:
#             return 0
#         sim = (len(set(tg.successors(a)).intersection(set(tg.successors(b)))))/\
#                                     (math.sqrt(len(set(tg.successors(a)))*len((set(tg.successors(b))))))    
#         return sim
#     except:
#         return 0

# # for followers 
# def cosine_for_followers(a,b):
#     try:
        
#         if len(set(tg.predecessors(a))) == 0  | len(set(tg.predecessors(b))) == 0:
#             return 0
#         sim = (len(set(tg.predecessors(a)).intersection(set(tg.predecessors(b)))))/\
#                                      (math.sqrt(len(set(tg.predecessors(a))))*(len(set(tg.predecessors(b)))))
#         return sim
#     except:
#         return 0


## 3. Ranking Measures

https://networkx.github.io/documentation/networkx-1.10/reference/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html

PageRank computes a ranking of the nodes in the graph G based on the structure of the incoming links.

<img src='PageRanks-Example.jpg' height=300px />

Mathematical PageRanks for a simple network, expressed as percentages. (Google uses a logarithmic scale.) Page C has a higher PageRank than Page E, even though there are fewer links to C; the one link to C comes from an important page and hence is of high value. If web surfers who start on a random page have an 85% likelihood of choosing a random link from the page they are currently visiting, and a 15% likelihood of jumping to a page chosen at random from the entire web, they will reach Page E 8.1% of the time. <b>(The 15% likelihood of jumping to an arbitrary page corresponds to a damping factor of 85%.) Without damping, all web surfers would eventually end up on Pages A, B, or C, and all other pages would have PageRank zero. In the presence of damping, Page A effectively links to all pages in the web, even though it has no outgoing links of its own.</b>

https://en.wikipedia.org/wiki/PageRank

### ` page rank is fairly based on complex graph theory, so we are not going to go deep here about various aspects of pagerank like 'alhpa value' etc.` 

In [9]:
if not os.path.isfile('datasets/pickle/page_rank.p'):
    pr = nx.pagerank(tg, alpha=0.85)
    pickle.dump(pr,open('datasets/pickle/page_rank.p','wb'))
else:
    pr = pickle.load(open('datasets/pickle/page_rank.p','rb'))

In [10]:
type(pr)

dict

In [11]:
print('min',min(list(pr.values())))
print('max',max(list(pr.values())))
print('mean',float(sum(pr.values())) / len(pr))

min 1.6556497245737814e-07
max 2.7098251341935827e-05
mean 5.615699699389075e-07


In [12]:
#for imputing to nodes which are not there in Train data
mean_pr = float(sum(pr.values())) / len(pr)
print(mean_pr)

5.615699699389075e-07


 # 4. Other Graph Features

## 4.1 Shortest path:

Getting Shortest path between twoo nodes, if nodes have direct path i.e directly connected then we are removing that edge and calculating path. 

In [13]:
def shortest_path(u,v,g):
    try:
        if not nx.has_path(g,u,v):
            return -1
        elif g.has_edge(u,v):
            g.remove_edge(u,v)
            p = nx.shortest_path_length(g,source=u,target=v)
            g.add_edge(u,v)
            return p
        else:
            return nx.shortest_path_length(g,source=u,target=v)
    except:
        return -1

## 4.2 Checking for same community

In [14]:
#getting weekly connected edges from graph 
wcc=list(nx.weakly_connected_components(tg))
def same_wcc(u,v):
    index = []
    if tg.has_edge(v,u):
        return 1
    if tg.has_edge(u,v):
            for i in wcc:
                if u in i:
                    index= i
                    break
            if (v in index):
                tg.remove_edge(u,v)
                if shortest_path(u,v,tg)==-1:
                    tg.add_edge(u,v)
                    return 0
                else:
                    tg.add_edge(u,v)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if u in i:
                    index= i
                    break
            if(v in index):
                return 1
            else:
                return 0

## 4.3 Adamic/Adar Index:
Adamic/Adar measures is defined as inverted sum of degrees of common neighbours for given two vertices.
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

In [15]:
#adar index
def adar_index(u,v):
    sum=0
    try:
        n=list(set(tg.successors(u)).intersection(set(tg.successors(v))))
        if len(n)!=0:
            for i in n:
                sum=sum+(1/np.log10(len(list(tg.predecessors(i)))))
            return sum
        else:
            return 0
    except:
        return 0

## 4.4 Is persion was following back:

In [16]:
def follow_back(u,v):
    try:
        if tg.has_edge(v,u):
            return 1
        else: 
            return 0 
    except:
        return 0 

## 4.5 Katz Centrality:
https://en.wikipedia.org/wiki/Katz_centrality

https://www.geeksforgeeks.org/katz-centrality-centrality-measure/
 Katz centrality computes the centrality for a node 
    based on the centrality of its neighbors. It is a 
    generalization of the eigenvector centrality. The
    Katz centrality for node `i` is
 
$$x_i = \alpha \sum_{j} A_{ij} x_j + \beta,$$
where `A` is the adjacency matrix of the graph G 
with eigenvalues $$\lambda$$.

The parameter $$\beta$$ controls the initial centrality and 

$$\alpha < \frac{1}{\lambda_{max}}.$$

In [21]:
if not os.path.isfile('datasets/pickle/katz.p'):
    katz = nx.katz.katz_centrality(tg,alpha=0.005,beta=1)
    pickle.dump(katz,open('datasets/pickle/katz.p','wb'))
else:
    katz = pickle.load(open('datasets/pickle/katz.p','rb'))

print('min',katz[min(katz, key=katz.get)])
print('max',katz[max(katz, key=katz.get)])
# print('mean',float(sum(katz.values())) / len(katz))

mean_katz = float(sum(katz.values())) / len(katz)
print(mean_katz)

min 0.0007313532484065916
max 0.003394554981699122
0.0007483800935562018


## 4.6 Hits Score
The HITS algorithm computes two numbers for a node. Authorities estimates the node value based on the incoming links. Hubs estimates the node value based on outgoing links.

https://en.wikipedia.org/wiki/HITS_algorithm

In [23]:
if not os.path.isfile('datasets/pickle/hits.p'):
    hits = nx.hits(tg, max_iter=100, tol=1e-08, nstart=None, normalized=True)
    pickle.dump(hits,open('datasets/pickle/hits.p','wb'))
else:
    hits = pickle.load(open('datasets/pickle/hits.p','rb'))

print('min',hits[0][min(hits[0], key=hits[0].get)])
print('max',hits[0][max(hits[0], key=hits[0].get)])
print('mean',float(sum(hits[0].values())) / len(hits[0]))

min 0.0
max 0.004868653378780953
mean 5.615699699344123e-07


# 5. Featurization

## 5. 1 Reading a sample of Data from both train and test